In [3]:
import torch

In [4]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda l: "".join(itos[i] for i in l)

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [10]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [81]:
torch.manual_seed(1337)
batch_size = 4
context_size = 8


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i : i + context_size] for i in ix])
    y = torch.stack([data[i + 1 : i + context_size + 1] for i in ix])
    return x, y

In [33]:
torch.manual_seed(1337)
xb, yb = get_batch("train")

tensor([ 76049, 234249, 934904, 560986])


In [70]:
import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)  # collapse so i can use NLL
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):  # gpt-like streaming
            logits, loss = self(idx)
            logits = logits[:, -1, :]  # just get what comes next
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)  # upd to next token
        return idx

In [130]:
model = BigramLanguageModel(vocab_size)
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), 100)[0].tolist()))


v?ejUk
gWghdAtoWzgTGTqPUfpT
vl!bRo TqZFJ-ekQMC.B$Bt$myNdneEEbamGyFhqVT!mzgCngdwMjrXS&
ScqbqVCNtsM d;


In [155]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [232]:
batch_size = 32
for epoch in range(10000):
    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(loss.item())

2.4384336471557617
2.6143569946289062
2.5687055587768555
2.5410969257354736
2.4855878353118896
2.4748616218566895
2.527994155883789
2.388068914413452
2.3501267433166504
2.491438150405884


In [238]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), 500)[0].tolist()))


S:
Herothin; awin; wh s he worurance me,'s y m thelalfe Henofrcouturyo
OUpenkithitof fods me s d, henoume;
POR:utout, Frded, m I y's moust omod d LIfr branourrn mshyoues mit here y, fes s st stho thiressissinot:
TENondd oury proror whano; be, cthe bore goullllacysshoft st citheasou fed wotispous?

LULor gutu f rknooldsef t r tale ng mike souirs
TUE:
LUEnd vinorersedindicale towilist

Fue? achilemoursond, g gh beistheathe f; t EN qu twndertimmy seatlo t h an s olare gen nt
SA ptono lie bery thato


In [261]:
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = F.softmax(wei, dim=-1)
out = wei @ x

out.shape

torch.Size([4, 8, 32])

In [274]:
q @ k.transpose(-2, -1)

tensor([[[ 1.3491e+00, -1.3778e+00, -4.9704e-01, -6.4773e-01, -6.5595e-01,
          -1.7690e-01,  5.7319e-01,  1.6617e+00],
         [ 7.7809e-01, -3.6348e-01,  3.6587e-01, -4.0361e-01,  3.8396e-01,
          -5.5295e-01, -1.2078e+00,  5.7024e-02],
         [-7.9892e-01, -1.9372e-01, -8.7293e-01, -2.1363e-01, -4.7672e-01,
           4.5359e-01, -1.0960e+00,  1.0909e+00],
         [-2.8941e+00,  3.6250e-01, -1.4500e+00,  1.0103e-01, -8.1625e-01,
           3.0137e-01,  1.1132e+00,  1.1987e+00],
         [ 1.2419e+00, -2.4641e+00,  3.1948e+00,  1.0834e+00,  1.2745e+00,
          -9.0531e-01,  9.6187e-01,  1.1861e+00],
         [ 2.4091e+00, -1.5802e+00,  1.3233e+00,  2.2658e-01,  1.5482e+00,
          -2.5287e+00,  2.5553e-01, -4.6723e-01],
         [-2.7087e+00,  4.3270e-01, -7.7406e-02,  8.3706e-01, -1.5908e-01,
           1.2910e+00, -1.0812e+00,  2.2230e+00],
         [ 4.3451e-01,  2.1320e-01, -5.7762e-01,  5.3235e-01,  7.6754e-01,
          -2.0091e+00,  1.5373e+00, -3.1043e+00]],